## In this notebook two models are tested:
### 1. Statistical model: This model considers all the price features and forecasts for 5 days ahead. Vector Autoregression Model is used for multivariate forecasting.
### 2. DL Model: This model has GRU layers and is used for univariate forecasting. The closing price is converted into a sliding window of past 20 days and mapped to 5 days ahead closing value for forecast.


In [13]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
# For reading stock data from yahoo
from pandas_datareader.data import DataReader
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU
import keras
from tensorflow.keras import initializers
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")
import random
import os
from statsmodels.tsa.api import VAR

In [23]:
config = {
    "companies": {
        "tickers": ['BNS', 'RY', 'TD', 'BMO', 'CM', 'C', 'JPM', 'IBN', 'WTBA', 'BAC', 'AXP',
       'PNC'],
    },
    "data": {
        "test_len": 5,
        "window": 25,
        "features": 1
    }, 
    "model": {
        "num_lstm_layers": 2,
        "lstm_size": 32,
        "dropout": 0.2,
    },
    "training": {
        "batch_size": 32,
        "epochs": 50
    }
}

In [7]:
def invert_transformation(X_train, pred):
    forecast = pred.copy()
    columns = X_train.columns
    for col in columns:
        forecast[str(col)+'_pred'] = X_train[col].iloc[-1] + forecast[str(col)+'_pred'].cumsum()
    return forecast

def statistical_model(df):
    X_train=df[:-5]
    X_test=df[-5:]
    X_diff=X_train.diff().dropna()
    mod = VAR(X_diff)
    res = mod.fit(maxlags=30, ic='aic')
    #print(res.summary())
    y_fitted = res.fittedvalues
    lag_order = res.k_ar
    input_data = X_diff.values[-lag_order:]
    pred = res.forecast(y=input_data, steps=5)
    pred = pd.DataFrame(pred, index=X_test.index, columns=X_test.columns + '_pred')
    output = invert_transformation(X_train, pred)
    #print(output)
    out=output["Close_pred"]
    #print(out)
    o1=out[-1:]
    o2=df["Close"][-1:]
    prediction=output["Close_pred"].values
    return prediction

## Overfitting is stopped using callbacks like early stopping and a validation split.

In [64]:
def get_data(ticker):
    df = DataReader(ticker, data_source='yahoo', start='2015-01-01', end=datetime.now())
    return df

def data_processing(config,df):
    d_c = df.filter(['Close'])
    data = d_c.values
    train_len = len(data)-config["data"]["test_len"]
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(data)
    train_data = scaled_data[0:int(train_len), :]
    x_train = []
    y_train = []
    for i in range(config["data"]["window"], len(train_data)):
        x_train.append(train_data[i-config["data"]["window"]:i, 0])
        y_train.append(train_data[i, 0])
    x_train, y_train = np.array(x_train), np.array(y_train)
    #print(x_train.shape)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    test_data = scaled_data[train_len - config["data"]["window"]: , :]
    x_test = []
    y_test = data[train_len:, :]
    for i in range(config["data"]["window"], len(test_data)):
        x_test.append(test_data[i-config["data"]["window"]:i, 0])
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))
    #print(x_train.shape)
    #print(x_test.shape)
    return x_train,y_train,x_test,y_test, scaler



def preprocess_dl(lag,steps_ahead,df):
    df = df.filter(['Close']).values
    scaler = MinMaxScaler(feature_range=(0,1))
    df = scaler.fit_transform(df)
    y=df[lag+steps_ahead-1:]
    y_train=y[:-test_len]
    y_test=y[-test_len:]
    x=series_to_supervised_dl(df,lag-1)
    x_train=x[:len(y_train)].values
    x_test=x[-test_len-steps_ahead:-steps_ahead].values
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
    return x_train,x_test,y_train,y_test,scaler



def model(config):
    os.environ['PYTHONHASHSEED'] = '42'
    np.random.seed(42)
    np.random.RandomState(42)
    random.seed(42)
    
    model = Sequential()
    model.add(GRU(128, return_sequences=True, input_shape= (config["data"]["window"], config["data"]["features"])))
    #model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(GRU(64, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    return model

def train_univariate(config,model,x_train,y_train,path):
    reduce_lr = keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
    checkpoint1 = keras.callbacks.ModelCheckpoint(path,
                                                     monitor='val_loss',
                                                     save_best_only=True,
                                                     mode='min')
    es=keras.callbacks.EarlyStopping(patience=5)


    model.compile(optimizer='adam', loss='mean_squared_error')
    history=model.fit(x_train,y_train,epochs=config["training"]["epochs"],validation_split=0.2,verbose=0,batch_size=config["training"]["batch_size"],callbacks=[reduce_lr,checkpoint1,es])


    
def series_to_supervised_dl(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg
    
    
def predict_dl(model,path,x_test,y_test,scaler):
    model.load_weights(path)
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)
    #print(valid)
    return predictions

def check_worth_increase(config,predictions,df):
    past_close=df.filter(["Close"])[-config["data"]["test_len"]-1:-config["data"]["test_len"]]
    #print(df.filter(["Close"]))
    #print("The predicted value for 5 days ahead is ",predictions[-1:], "the actual value is ",df.filter(["Close"])[-1:])
    if predictions[-1:] > past_close.values[0]:
        print("future value {} greater than past value {}".format(predictions[-1:],past_close.values))
        return 1
    else:
        print("future value {} lesser than past value {}".format(predictions[-1:],past_close.values))
        return 0
    
    
def true_worth_increase(config,df):
    past_close=df.filter(["Close"])[-config["data"]["test_len"]-1:-config["data"]["test_len"]]
    #print(df.filter(["Close"]))
    #print("The predicted value for 5 days ahead is ",predictions[-1:], "the actual value is ",df.filter(["Close"])[-1:])
    if df.filter(["Close"])[-1:].values > past_close.values:
        print("future actual value {} greater than past value {}".format(df.filter(["Close"])[-1:],past_close.values))
        return 1
    else:
        print("future actual value {} lesser than past value {}".format(df.filter(["Close"])[-1:],past_close.values))
        return 0   
    
def test(pred,true):
    rmse = np.sqrt(np.mean(((pred - true) ** 2)))
    mae = mean_absolute_error(pred, true)
    mape= np.mean(mae/true) *100
    return rmse, mae, mape
  
def sliding_data(df,index):
    df=df[:-index]
    return df

# Training and Testing the models
## A sliding window testing technique is used in which the models are trained and tested 10 times and data is sliced in a way to test for 5 days ahead forecast.

## Thus the model is tested on 50 data points which is the data for 10 weeks

In [28]:

DL,STAT=[],[]
lag = 25
steps_ahead = 5
n_features = 6
test_len=5

for tick in config["companies"]["tickers"]:
    print(" \n \n For:",tick)
    df=get_data(tick)
    metric_dl, metric_stat=[],[]
    for idx in range(5,50,5):  
        true=df["Close"][-test_len:].values
        path="bestweight"+tick+".hdf5" 
        x_train,x_test,y_train,y_test,scaler=preprocess_dl(lag,steps_ahead,df)
        #print(x_train.shape)
        #print(y_train.shape)
        model1=model(config)
        print("Training...")
        train_univariate(config,model1,x_train,y_train,path)
        predictions=predict_dl(model1,path,x_test,y_test,scaler)
        rmse, mae, mape=test(predictions,true)
        metric_dl.append([rmse, mae, mape])
        pred=statistical_model(df)
        rmse, mae, mape=test(pred,true)
        metric_stat.append([rmse, mae, mape])
        df=sliding_data(df,idx)
    df_DL = DataFrame(metric_dl,columns=['RMSE','MAE','MAPE'])
    print(np.mean(df_DL["MAPE"]))
    df_STAT = DataFrame(metric_stat,columns=['RMSE','MAE','MAPE'])
    print(np.mean(df_STAT["MAPE"]))
    STAT.append(np.mean(df_STAT["MAPE"]))
    DL.append(np.mean(df_DL["MAPE"]))

 
 
 For: BNS
Training...
Training...
Training...
Training...
Training...
Training...
Training...
Training...
Training...
2.1967088384472273
1.4477498917343106
 
 
 For: RY
Training...
Training...
Training...
Training...
Training...
Training...
Training...
Training...
Training...
2.29872083478817
1.1580047450664446
 
 
 For: TD
Training...
Training...
Training...
Training...
Training...
Training...
Training...
Training...
Training...
2.406784667147635
1.0908419954294595
 
 
 For: BMO
Training...
Training...
Training...
Training...
Training...
Training...
Training...
Training...
Training...
2.8068312532909467
1.350441816536779
 
 
 For: CM
Training...
Training...
Training...
Training...
Training...
Training...
Training...
Training...
Training...
2.844227486774407
1.2829042912326665
 
 
 For: C
Training...
Training...
Training...
Training...
Training...
Training...
Training...
Training...
Training...
4.059731016825612
2.83471534073952
 
 
 For: JPM
Training...
Training...
Training...
Tra

## The values below are the Mean Absolute Percentage Error values obtained for all the banks when tested using the sliding window testing 10 times for 5 values for the Statistical Model

In [39]:
STAT_mape=pd.DataFrame(STAT).T
STAT_mape.columns=config["companies"]["tickers"]
STAT_mape

,BNS,RY,TD,BMO,CM,C,JPM,IBN,WTBA,BAC,AXP,PNC
0,1.44775,1.158005,1.090842,1.350442,1.282904,2.834715,2.227976,2.244519,2.861901,2.821382,2.945433,2.856769


## The average MAPE of the this statistical model is around 2 %

In [32]:
np.asarray(STAT).mean()

2.093553150095195

## The values below are the Mean Absolute Percentage Error values obtained for all the banks when tested using the sliding window testing 10 times for 5 values for the DL Model

In [40]:
DL_mape=pd.DataFrame(DL).T
DL_mape.columns=config["companies"]["tickers"]
DL_mape

,BNS,RY,TD,BMO,CM,C,JPM,IBN,WTBA,BAC,AXP,PNC
0,2.196709,2.298721,2.406785,2.806831,2.844227,4.059731,2.787255,3.103551,3.620858,4.004264,3.202899,3.64671


## The average MAPE of the this DL model is around 3 %

In [41]:
np.asarray(DL).mean()

3.081545138138807